## Setup

In [ ]:
import os

# importing scripting dependencies
import boa
from eth_account import Account

In [ ]:
from dotenv import load_dotenv

load_dotenv()
PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")
# RPC_URL = "https://sepolia.optimism.io"
RPC_URL = "https://sepolia.base.org"
# RPC_URL = "https://mainnet.optimism.io"

## Select mode
Production mode cell is intentionally executed first to avoid deploying things by accident. If one runs all the cells sequentially things will be executed in forking mode.

To deploy in production mode, manually skip the execution of the fork mode cell.

In [ ]:
# production mode (to deploy on ethereum)
deployer = Account.from_key(PRIVATE_KEY)
eth_env = boa.set_network_env(RPC_URL)
# boa.set_env(eth_env)

# this automatically sets the eoa as the deployer
boa.env.add_account(deployer)
print(f"Deploying with {deployer.address}")

# I. Deployment

### 2. LZContract

In [ ]:
# II. Then deploy the DepositLimitModule
contract_deployer = boa.load_partial("../contracts/LZContract.vy")

deposit_limit = contract_deployer(
    "0x4200000000000000000000000000000000000015",  # l1 block precompile
)

print(f"Deposit limit module deployed at {deposit_limit.address}")

# II. Post-deployment spam

In [ ]:
from web3 import Web3
import os
import time
import logging
from datetime import datetime

# Configure logging
log_filename = f"op_monitor_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler(log_filename), logging.StreamHandler()],
)

# RPC endpoints
NETWORKS = {
    # 'optimism': 'https://sepolia.optimism.io',
    # 'base': 'https://sepolia.base.org'
    "OP": "https://mainnet.optimism.io"
}

CONTRACT_ADDRESS = "0x25A9a298494dB90851633E4D74e660C996379Ecc"

try:
    pk = os.environ["WEB3_TESTNET_PK"]
except KeyError:
    raise EnvironmentError("Please set WEB3_TESTNET_PK environment variable")

ABI = """[{"name":"BlockMismatch","inputs":[{"name":"block_number","type":"uint64","indexed":false},{"name":"stored_hash","type":"bytes32","indexed":false},{"name":"oracle_hash","type":"bytes32","indexed":false},{"name":"stored_timestamp","type":"uint64","indexed":false},{"name":"oracle_timestamp","type":"uint64","indexed":false}],"anonymous":false,"type":"event"},{"stateMutability":"nonpayable","type":"function","name":"fetch_latest_block","inputs":[],"outputs":[]},{"stateMutability":"nonpayable","type":"function","name":"get_block_hash","inputs":[{"name":"block_number","type":"uint64"}],"outputs":[{"name":"","type":"bytes32"}]},{"stateMutability":"nonpayable","type":"function","name":"get_block_timestamp","inputs":[{"name":"block_number","type":"uint64"}],"outputs":[{"name":"","type":"uint64"}]},{"stateMutability":"view","type":"function","name":"peek_l1block_number","inputs":[],"outputs":[{"name":"","type":"uint64"}]},{"stateMutability":"view","type":"function","name":"l1_blocks","inputs":[{"name":"arg0","type":"uint64"}],"outputs":[{"name":"","type":"tuple","components":[{"name":"block_hash","type":"bytes32"},{"name":"block_timestamp","type":"uint64"}]}]},{"stateMutability":"view","type":"function","name":"last_fetched_block","inputs":[],"outputs":[{"name":"","type":"uint64"}]},{"stateMutability":"nonpayable","type":"constructor","inputs":[{"name":"l1block_precompile_address","type":"address"}],"outputs":[]}]"""


def setup_web3_and_contracts():
    networks = {}
    for name, rpc in NETWORKS.items():
        try:
            w3 = Web3(Web3.HTTPProvider(rpc))
            account = w3.eth.account.from_key(pk)
            contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)
            networks[name] = {"w3": w3, "contract": contract, "account": account}
            logging.info(f"Connected to {name}: {rpc}")
        except Exception as e:
            logging.error(f"Failed to connect to {name}: {e}")
    return networks


def monitor_blocks(networks):
    while True:
        t0 = time.time()
        for name, network in networks.items():
            try:
                func = network["contract"].functions.fetch_latest_block()
                tx = func.build_transaction(
                    {
                        "from": network["account"].address,
                        "nonce": network["w3"].eth.get_transaction_count(
                            network["account"].address
                        ),
                    }
                )
                tx["gas"] = int(5 * network["w3"].eth.estimate_gas(tx))
                signed_tx = network["w3"].eth.account.sign_transaction(
                    tx, private_key=network["account"].key
                )
                tx_hash = network["w3"].eth.send_raw_transaction(signed_tx.raw_transaction)

                logging.info(f"{name.upper()}: Tx: {tx_hash.hex()}")

            except Exception as e:
                logging.error(f"{name.upper()} Error: {str(e)}")
                continue

        time.sleep(max(0, 5 - (time.time() - t0)))


def main():
    logging.info("Starting L1 Block Oracle monitoring")
    networks = setup_web3_and_contracts()

    try:
        monitor_blocks(networks)
    except KeyboardInterrupt:
        logging.info("Monitoring stopped by user")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        raise


if __name__ == "__main__":
    main()